# Automação Web 

#### Desafio: 

Trabalhamos numa empresa de importação e o preço dos nossos produtos é vinculado à cotação de:
- Dólar
- Euro
- Ouro

Precisamos ir à internet e de forma automática verificar a cotação desses 3 itens e conseguirmos assim asaber quanto podemos cobrar pelos nossos produtos, considerando uma margem de lucro que temos na nossa Base de dados.

Base de Dados: https://docs.google.com/spreadsheets/d/14i0xeIatBrSAl_SgRH-kNz7SPaUKRtsM/edit?usp=sharing&ouid=111472310099212768630&rtpof=true&sd=true

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Para abrir o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

# abrir um navegador
navegador = webdriver.Chrome()
# caso queira deixar na mesma pasta do seu código
# navegador = webdriver.Chrome("chromedriver.exe")


navegador.get("https://www.google.com/")

navegador.find_element_by_xpath(
    '//*[@id="L2AGLb"]/div').click()

# Passo 1: ir buscar a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

C:\Users\ITCenter\AppData\Local\Temp\ipykernel_12308\1270904271.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  navegador.find_element_by_xpath(


0.9601599999999999
1.04135
298.20


### Agora vamos atualizar a nossa base de Dados de preços com as novas cotações

- Importar a Base de dados

In [6]:
# Passo 4: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

ModuleNotFoundError: No module named 'pandas'

- Atualizando os preços e o cálculo do Preço Final

In [4]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.140600,5140.548594,1.40,7196.768032
1,Carro Renault,4500.00,Euro,5.403439,24315.474951,2.00,48630.949902
2,Notebook Dell,899.99,Dólar,5.140600,4626.488594,1.70,7865.030610
3,IPhone,799.00,Dólar,5.140600,4107.339400,1.70,6982.476980
4,Carro Fiat,3000.00,Euro,5.403439,16210.316634,1.90,30799.601605
5,Celular Xiaomi,480.48,Dólar,5.140600,2469.955488,2.00,4939.910976
6,Joia 20g,20.00,Ouro,305.960000,6119.200000,1.15,7037.080000


### Agora vamos exportar a nova base de preços atualizada

In [5]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)